In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az

In [41]:
model = pm.Model()

with model:
    earthquake = pm.Bernoulli('E', 0.05)
    fire = pm.Bernoulli('F', pm.math.switch(earthquake, 0.03, 0.01))
    alarm_p = pm.math.switch(earthquake, pm.math.switch(fire, 0.98, 0.02), pm.math.switch(fire, 0.95, 0.0001))
    alarm = pm.Bernoulli('A', p=alarm_p)
    trace = pm.sample(20000)

/home/ina/.local/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [E, F, A]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 12 seconds.
The number of effective samples is smaller than 10% for some parameters.


In [42]:
dictionary = {
              'earthquake': trace['E'].tolist(),
              'fire': trace['F'].tolist(),
              'alarm': trace['A'].tolist(),
              }
df = pd.DataFrame(dictionary)

In [43]:
p_earthquake = df[((df['earthquake'] == 1) & (df['alarm'] == 1))].shape[0] / df[df['alarm'] == 1].shape[0]
print('P(E=1|A=1) =', p_earthquake)

p_earthquake = df[((df['fire'] == 1) & (df['alarm'] == 0))].shape[0] / df.shape[0]
print('P(F=1,A=0) =', p_earthquake)


P(E=1|A=1) = 0.21967526265520534
P(F=1,A=0) = 0.000525
